# DownloadTorrent TO GDrive 

**Important Note:** Runtime already in GPU mode(no need to change):
> Run the first cell/command -> Run the second cell to connect your any google Drive account manually(download path) ->  Run third cell.You will able to see upload option.Simply upload your torrent file and then tap 'Shift+Enter'-> If you want to download torrent via magnet url run fourth cell.Otherwise leave it(dont do anything) -> Now go to 'Start Download' area.Run that cell.Your file will be downloading perfectly.See the download bar bellow.! -> To see the downloaded file.Simply go to Google Drive (that account you mounted here few min ago/in the second cell).In your Google Drive there will be a new folder name as 'torrent'.That's the download path of your downloaded file. :) 

### Install libtorrent and Initialize Session

In [1]:
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Run the cell and Wait for the installation.



### Mount Google Drive
Need to mount Google Drive for the downloading patch.No worries about Data Loss issue.Cause it's not a third-party proccess. :)

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Run this cell.After running you will get an url.Open that url in a new tab.

Enter your authorization code and tap 'Shift+Enter':
··········
Mounted at /content/drive


### Add From Torrent File
At first run this cell then you will get upload option.Simply upload your torrent file and tap 'Shift+Enter'

In [0]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### Add From Magnet Link
Run this cell to add more files as many times as you want via Magnet url.Otherwise leave empty!

In [3]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Enter Magnet Link Or Type Exit: Just paste the Magnet url and tap 'Shift+Enter'


### Start Download


In [4]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)
while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() 
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…